# Saving field data to hdf5 files

In [1]:
import dolfin as df
from finmag import Simulation as Sim
from finmag.energies import Exchange, Zeeman, DMI
from finmag.util.consts import mu0

[2015-10-16 11:45:09] INFO: Finmag logging output will be appended to file: '/home/mb4e10/.finmag/global.log'
[2015-10-16 11:45:09] DEBUG: Building modules in 'native'...
[2015-10-16 11:45:09] DEBUG: FinMag          16ac4753324284a325002b66f291256813e877d0
[2015-10-16 11:45:09] DEBUG: Dolfin          1.6.0                Matplotlib      lazily loaded       
[2015-10-16 11:45:09] DEBUG: Numpy           1.9.2                Scipy           0.15.1              
[2015-10-16 11:45:09] DEBUG: IPython         3.2.0                Python          2.7.6               
[2015-10-16 11:45:09] DEBUG: Paraview        4.0.1-1ubuntu1       Sundials        2.5.0               
[2015-10-16 11:45:09] DEBUG: Boost-Python    <unknown>            Linux           Ubuntu 14.04.3 LTS  
[2015-10-16 11:45:09] DEBUG: Registering debug signal handler. Press Ctrl-Z any time to stop execution and jump into the debugger.


First, we create the simulation object and initialise the magnetisation:

In [2]:
mesh = df.RectangleMesh(df.Point(0, 0), df.Point(100, 100), 10, 10)

Ms = 1e6
A = 1e-11
D = 3e-3

sim = Sim(mesh, Ms, unit_length=1e-9)

sim.add(Exchange(A))
sim.add(DMI(D))
sim.add(Zeeman((0, 0, 0.1/mu0)))
sim.set_m((1, 0, 0))

[2015-10-16 11:45:11] INFO: Finmag logging output will be written to file: '/home/mb4e10/finmag/doc/ipython_notebooks_src/unnamed.log' (any old content will be overwritten).
[2015-10-16 11:45:11] DEBUG: Creating DataWriter for file 'unnamed.ndt'
[2015-10-16 11:45:11] INFO: Creating Sim object name='unnamed', instance_id=0 (rank=0/1).
[2015-10-16 11:45:11] DEBUG:    Total number of Sim objects in this session: 1
[2015-10-16 11:45:11] INFO: <Mesh of topological dimension 2 (triangles) with 121 vertices and 200 cells, ordered>
/usr/local/lib/python2.7/dist-packages/aeon/timer.py:35: UserWarning: You are nesting measurements in __init__::LLG.
  warnings.warn("You are nesting measurements in {}::{}.".format(name, group))
[2015-10-16 11:45:11] DEBUG: Creating LLG object.
[2015-10-16 11:45:11] DEBUG: Creating Exchange object with method box-matrix-petsc, in Jacobian.
[2015-10-16 11:45:11] DEBUG: Adding interaction Exchange to simulation.
[2015-10-16 11:45:11] DEBUG: Creating DMI object with m

## Saving field

Now, we run the simulation in uniform timesteps ($\Delta t = 20 \,\text{ps}$) for 0.5 ns and in every time step, we save the magnetisation field.

In every step, we provide both the name and time at which the magnetisation is saved.

In [3]:
import numpy as np

filename = 'magnetisation'

T = 0.1e-9
dt = 20e-12

sim.reset_time(0)

t_array = np.arange(0, T+dt/2, dt)
for t in t_array:
    sim.run_until(t)
    sim.llg.m_field.save_hdf5(filename, t=sim.t/1e-9)
    
sim.llg.m_field.close_hdf5()

[2015-10-16 11:45:13] INFO: Creating integrator with backend sundials and arguments {'t0': 0}.
[2015-10-16 11:45:13] INFO: Simulation will run until t = 0 s.
[2015-10-16 11:45:13] INFO: Simulation has reached time t = 0 s.
[2015-10-16 11:45:13] INFO: Simulation will run until t = 2e-11 s.
/usr/local/lib/python2.7/dist-packages/aeon/timer.py:35: UserWarning: You are nesting measurements in compute_field::DMI.
  warnings.warn("You are nesting measurements in {}::{}.".format(name, group))
/usr/local/lib/python2.7/dist-packages/aeon/timer.py:35: UserWarning: You are nesting measurements in compute_field::Exchange.
  warnings.warn("You are nesting measurements in {}::{}.".format(name, group))
[2015-10-16 11:45:13] INFO: Simulation has reached time t = 2e-11 s.


Debug: (0/1) opening file magnetisation.h5
Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json

[2015-10-16 11:45:13] INFO: Simulation will run until t = 4e-11 s.
[2015-10-16 11:45:13] INFO: Simulation has reached time t = 4e-11 s.
[2015-10-16 11:45:13] INFO: Simulation will run until t = 6e-11 s.
[2015-10-16 11:45:13] INFO: Simulation has reached time t = 6e-11 s.



Debug: (0/1) writing json file magnetisation.json
Debug: (0/1) writing json file magnetisation.json

[2015-10-16 11:45:13] INFO: Simulation will run until t = 8e-11 s.
[2015-10-16 11:45:13] INFO: Simulation has reached time t = 8e-11 s.



Debug: (0/1) writing json file magnetisation.json

[2015-10-16 11:45:13] INFO: Simulation will run until t = 1e-10 s.
[2015-10-16 11:45:13] INFO: Simulation has reached time t = 1e-10 s.



Debug: (0/1) writing json file magnetisation.json


We can now see that both h5 and json files are created.

In [4]:
!ls *.json
!ls *.h5

magnetisation.json
magnetisation.h5


## Loading field

An object for reading the data from saved hdf5 file can be created as:

In [5]:
from dolfinh5tools import openh5
h5file = openh5('magnetisation', field_name='m', mode='r')

Now, the saved fields list as well as the times at which these fields were saved can be read:

In [6]:
saved_fields = h5file.fields()

for field in saved_fields:
    print 'Field:', field
    print 'Saved at times: ', h5file.times(field_name=field)

Field: m
Saved at times:  [0.0, 0.019999999999999997, 0.039999999999999994, 0.06, 0.07999999999999999, 0.09999999999999998]


A particular field (as dolfin function) at a certain time can be loaded as:

In [7]:
m_loaded = h5file.read(t=0.06)
m_loaded.vector().array()

array([  1.09775439e-01,  -2.80839261e-01,  -6.79544452e-01,
        -8.46365564e-01,  -8.58054012e-01,  -8.38819870e-01,
        -8.38157307e-01,  -8.59638416e-01,  -9.03675348e-01,
        -9.54891921e-01,  -9.60365618e-01,   4.22114876e-01,
         2.26649618e-01,  -7.10651640e-02,  -3.10915117e-01,
        -4.23232856e-01,  -4.51488487e-01,  -4.66115630e-01,
        -4.96961212e-01,  -5.41093265e-01,  -5.69638387e-01,
        -6.31937752e-01,   5.99994163e-01,   4.99941893e-01,
         3.22972669e-01,   1.54229268e-01,   4.47356416e-02,
        -1.27894211e-02,  -4.21539418e-02,  -7.74036265e-02,
        -2.05343558e-01,  -3.11829892e-01,  -3.69322850e-01,
         6.74541213e-01,   6.32877253e-01,   5.15926399e-01,
         4.01483244e-01,   3.25321674e-01,   2.63329165e-01,
         2.30668279e-01,   2.58378671e-01,   6.00991616e-02,
        -1.70088844e-01,  -2.72952841e-01,   7.97217121e-01,
         7.46530623e-01,   6.37661176e-01,   5.50559474e-01,
         4.98449812e-01,